## 4.3 Sequential MNIST
### RNN
`cell = tf.nn.rnn_cell.LSTMCell(num_units=100, use_peepholes=True)`  
で個別の層の設定終わり。  
`output, state = tf.nn.dynamic_rnn(cell=cell, inputs=x, dtype=tf.float32)`
で層全体の設定終わりで出力。  
多層のRNNは1つ目のセルを何個か作って  
`cell = tf.nn.rnn_cell.MultiRnnCell(cells=cells)`  
のようにがっちゃんこする関数に渡すだけ。簡単すぎる。  
ちなみに`tf.nn.dynamic_rnn`の他に`tf.nn.static_rnn`もあるけど、前者のほうが実装が楽だしモデル構築も高速なので本の中では前者を採用してるらしい。詳しい違いはリファレンスなり本文を確認されたし。  
追記：[Qiitaに記事あった](https://qiita.com/suckgeun/items/c9c400f37979b3d4ca89)

### データ
MISTの画像はグレースケール（チャンネル数が１）なので、[高さ、幅、チャンネル]の3階テンソルを[高さ、幅]の２階テンソルとして扱うことができる。  
また、画像サイズが固定なので、１行の画素データ群の時系列の入力として見ることができる。  
（１行目を`t=1`、２行目を`t=2`とみなせる）  
  
### 注意点
MNISTはRNNで学習させてもあまりメリットがなく、速度も精度も劣るが、データがある状態で実装を行い、RNNの実装そのものに慣れるのに最適なデータセットなので今回はMNISTでRNNを実装している。
  
（あとで実装するWord2Vecを見ればわかる通り、形態素解析などのデータの前処理が大変なので、いきなりそれをするとRNNに集中できない）  


In [1]:
#!/usr/bin/env python
# coding=utf-8

import tensorflow as tf

def mnist_rnn():
    # importing MNIST data
    from tensorflow.examples.tutorials.mnist import input_data
    mnist = input_data.read_data_sets('mnist_data/', one_hot=True)

    # input data
    num_seq = 28
    num_input = 28

    # need to transform binary data into [batch_size, num_seq, num_input]
    x = tf.placeholder(tf.float32, [None, 784])
    input = tf.reshape(x, [-1, num_seq, num_input])

    # setting LSTM cells with 128 units
    # piled up to three stacks
    stacked_cells = []
    for i in range(3):
        stacked_cells.append(tf.nn.rnn_cell.LSTMCell(num_units=128))
    cell = tf.nn.rnn_cell.MultiRNNCell(cells=stacked_cells)

    # Creating RNN
    # about outputs:
    # https://stackoverflow.com/questions/48238113/tensorflow-dynamic-rnn-state
    outputs, _ = tf.nn.dynamic_rnn(cell=cell, inputs=input, dtype=tf.float32)

    last_output = outputs[:, -1, :]

    w = tf.Variable(tf.truncated_normal([128, 10], stddev=0.1))
    b = tf.Variable(tf.zeros([10]))

    out = tf.nn.softmax(tf.matmul(last_output, w) + b)

    # after creating rnn, train and test as usual
    y = tf.placeholder(tf.float32, [None, 10])

    loss = tf.reduce_mean(-tf.reduce_sum(y * tf.log(out), axis=[1]))

    train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

    correct = tf.equal(tf.argmax(out, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

    init = tf.global_variables_initializer()

    with tf.Session() as sess:
        sess.run(init)

        test_images = mnist.test.images
        test_labels = mnist.test.labels

        for i in range(1000):
            step = i+1
            train_images, train_labels = mnist.train.next_batch(50)
            sess.run(train_step, feed_dict={x:train_images ,y:train_labels})

            if step % 100 == 0:
                acc_val = sess.run( accuracy, feed_dict={x:test_images, y:test_labels})
                print('Step {}: accuracy = {}'.format(step, acc_val))

/home/tk/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
mnist_rnn()

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting mnist_data/t10k-images-idx3-ubyte.gz
Extracting mnist_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Step 100: accuracy = 0.33640000224113464
Step 200: accuracy = 0.5544999837875366
Step 300: accuracy = 0.6705999970436096
Step 400: accuracy = 0.6929000020027161
Step 500: accuracy = 0.7354000210762024
Step 600: accuracy = 0.8342000246047974
Step 700: accuracy = 0.864300012588501
Step 800: accuracy = 

## Word2Vec
### date_set.py

以下のコマンドでMeCabとnatto-pyのインストールを行っておくこと。

In [3]:
#!/usr/bin/env python
# coding=utf-8

import glob
import re
import collections
import random

import numpy as np
from natto import MeCab


# loading files
class DataSet(object):

    def __init__(self, data_dir, max_vocab):

        # 全データセットのファイルパスを取得
        file_pathes = []
        for file_path in glob.glob(data_dir+'*'):
            file_pathes.append(file_path)

        # ファイルを読み込み
        row_documents = [self._read_docment(file_path)
                         for file_path in file_pathes]
        # 必要な部分だけ抽出
        documents = [self._preprocessing(document)
                     for document in row_documents]
        # 形態素解析
        splited_documents = [self._morphological(document)
                             for document in documents]

        words = []
        for word_list in splited_documents:
            words.extend(word_list)

        # データセット作成
        self.id_sequence, self.word_frequency, self.w_to_id, self.id_to_w \
            = self._build_data_sets(words, max_vocab)
        print('Most common words (+UNK)', self.word_frequency[:5])
        print('Sample data.')
        print(self.id_sequence[:10])
        print([self.id_to_w[i] for i in self.id_sequence[:10]])
        self.data_index = 0

    # ファイルの読み込み
    def _read_docment(self, file_path):
        with open(file_path, 'r', encoding='sjis') as f:
            return f.read()

    # ヘッダなどの不要データを前処理。必要な部分だけを返す。
    def _preprocessing(self, document):

        lines = document.splitlines()
        processed_line = []

        horizontal_count = 0

        for line in lines:

            # ヘッダーは読み飛ばす
            if horizontal_count < 2:
                if line.startswith('-------'):
                    horizontal_count += 1
                continue
            # フッターに入る行になったらそれ以降は無視
            if line.startswith('底本：'):
                break

            line = re.sub(r'《.*》', '', line)  # ルビを除去
            line = re.sub(r'［.*］', '', line)   #脚注を除去
            line = re.sub(r'[!-~]', '', line)  # 半角記号を除去
            line = re.sub(r'[︰-＠]', '', line)  # 全角記号を除去
            line = re.sub('｜', '', line)  # 脚注の始まりを除去

            processed_line.append(line)

        return ''.join(processed_line)

    # 形態素解析
    def _morphological(self, document):

        word_list = []
        # MeCabの形態素解析結果のフォーマット
        with MeCab('-F%f[0],%f[1],%f[6]') as mcb:
            for token in mcb.parse(document, as_nodes=True):
                features = token.feature.split(',')
                # 名詞（一般）動詞（自立）、形容詞（自立）以外は除外
                if features[0] == '名詞' \
                        and features[1] == '一般' \
                        and features[2] != '':
                    word_list.append(features[2])
                if features[0] == '動詞' \
                        and features[1] == '自立' \
                        and features[2] != '':
                    word_list.append(features[2])
                if features[0] == '形容詞' \
                        and features[1] == '自立' \
                        and features[2] != '':
                    word_list.append(features[2])
        return word_list

    # 辞書作成
    def _build_data_sets(self, words, max_vocab):

        # 単語出現回数を解析。
        # 出現数が少ない単語をUnknown wordとしてひとくくりに扱う
        word_frequency = [['UNW', -1]]
        word_frequency.extend(collections.Counter(words)
                              .most_common(max_vocab - 1))
        # 単語=>IDの辞書
        w_to_id = dict()
        for word, _ in word_frequency:
            w_to_id[word] = len(w_to_id)
        # 形態素解析した文章を単語IDの並びに変換
        id_sequence = list()
        unw_count = 0
        for word in words:
            # UNK処理
            if word in w_to_id:
                index = w_to_id[word]
            else:
                index = 0
                unw_count += 1
            id_sequence.append(index)
        word_frequency[0][1] = unw_count
        # 単語ID=>単語の辞書
        id_to_w = dict(zip(w_to_id.values(), w_to_id.keys()))
        return id_sequence, word_frequency, w_to_id, id_to_w

    # num_skip:１つの入力をどれだけ再利用するか
    # skip_window: 左右何語までを正解対象にするか
    def create_next_batch(self, batch_size, num_skips, skip_window):

        assert batch_size % num_skips == 0
        # 一つの入力の再利用回数が対象範囲全件を超えてはならない
        assert num_skips <= 2 * skip_window
        inputs = np.ndarray(shape=(batch_size), dtype=np.int32)
        labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)

        span = 2 * skip_window + 1
        buffer = collections.deque(maxlen=span)
        # データセットが1週しそうならindexを最初にもどす
        if self.data_index + span > len(self.id_sequence):
            self.data_index = 0
        # 初期のqueueを構築(window内の単語をすべて格納)
        buffer.extend(self.id_sequence[self.data_index:self.data_index+span])
        self.data_index += span

        for i in range(batch_size // num_skips):
            # 中心は先に正解データから除外
            target = skip_window
            targets_to_avoid = [skip_window]
            for j in range(num_skips):
                # すでに選ばれている物以外から正解データのインデックスを取得
                while target in targets_to_avoid:
                    target = random.randint(0, span - 1)
                # 次回以降targetにならないように
                targets_to_avoid.append(target)
                # 入力値になるのはbufferの中心
                inputs[i * num_skips + j] = buffer[skip_window]
                # ランダムに指定した周辺単語が正解データに
                labels[i * num_skips + j, 0] = buffer[target]

            # 次に入れる単語がデータセットにない場合はbufferには最初の値を入力
            if self.data_index == len(self.id_sequence):
                buffer = self.id_sequence[:span]
                self.data_index = span
            else:
                # bufferに次の単語を追加してindexを1進める
                buffer.append(self.id_sequence[self.data_index])
                self.data_index += 1
        # 最後の方のデータが使われないことを避けるために少しだけindexを元に戻す
        self.data_index = (self.data_index + len(self.id_sequence) - span) \
            % len(self.id_sequence)

        return inputs, labels

### word2vec.py
#### FLAGS
`tf.app.flags.FLAGS`を利用すれば、`argparse`のようにプログラムを実行する際に引数を用いることができるようになる。  
  
なぜ必要か？：  
ディープラーニングではハイパーパラメーターをいくつも設定する必要があり、それを変更する際にプログラムをいちいち変更するのは面倒なので、ある程度ハイパーパラメーターが多くなった場合にはFLAGSを定義して引数を受け取れるようにするのが便利。  
  
#### 参考リンク  
- [What's the purpose of tf.app.flags in TensorFlow?
](https://stackoverflow.com/questions/33932901/whats-the-purpose-of-tf-app-flags-in-tensorflow)
- [TensorFlow – tf.app.flags.FLAGS（ファイル実行時にパラメタを付与できるようにする）
](https://endoyuta.com/2017/01/20/tensorflow-tf-app-flags-flags%EF%BC%88%E3%83%95%E3%82%A1%E3%82%A4%E3%83%AB%E5%AE%9F%E8%A1%8C%E6%99%82%E3%81%AB%E3%83%91%E3%83%A9%E3%83%A1%E3%82%BF%E3%82%92%E4%BB%98%E4%B8%8E%E3%81%A7%E3%81%8D/)

In [4]:
#!/usr/bin/env python
# coding=utf-8

import tensorflow as tf
import math
from data_set import *
FLAGS = tf.app.flags.FLAGS
tf.app.flags.DEFINE_string('data_dir', 'data/', "Data set directory.")
tf.app.flags.DEFINE_string('log_dir', 'logs/', "Log directory.")
tf.app.flags.DEFINE_integer('max_vocab', 2000, "Max Vocablary size.")
tf.app.flags.DEFINE_integer('skip_window', 2,
                            "How many words to consider left and right.")
tf.app.flags.DEFINE_integer('num_skips', 4,
                            "How many times to reuse input to get labels.")
tf.app.flags.DEFINE_integer('embedding_size', 64,
                            "Dimension of the embedding vector.")
tf.app.flags.DEFINE_integer('num_sumpled', 64,
                            "Number of negative examples to sample.")
tf.app.flags.DEFINE_integer('num_step', 10000, "Train step.")
tf.app.flags.DEFINE_integer('batch_size', 64, "Batch size.")
tf.app.flags.DEFINE_float('learning_rate', 0.1, "Learning rate." )
tf.app.flags.DEFINE_bool('create_tsv', True, "Create words.tsv or not." )


def main(argv):

    # データセットオブジェクトを作成
    data = DataSet(FLAGS.data_dir, FLAGS.max_vocab)

    # Embeddingsように使うラベルをtsv形式で保存
    if FLAGS.create_tsv:
        sorted_dict = sorted(data.w_to_id.items(), key=lambda x: x[1])
        words = ["{word}\n".format(word=x[0]) for x in sorted_dict]
        with open(FLAGS.log_dir+"words.tsv", 'w', encoding="utf-8") as f:
            f.writelines(words)
        print("Embeddings metadata was saved to "+FLAGS.log_dir+"/words.tsv")

    batch_size = FLAGS.batch_size
    embedding_size = FLAGS.embedding_size
    vocab_size = len(data.w_to_id)
    # placeholderの定義
    inputs = tf.placeholder(tf.int32, shape=[batch_size])
    correct = tf.placeholder(tf.int32, shape=[batch_size, 1])

    word_embedding = tf.Variable(tf.random_uniform(
        [vocab_size, embedding_size], -1.0, 1.0), name='word_embedding')
    embed = tf.nn.embedding_lookup(word_embedding, inputs)
    w_out = tf.Variable(tf.truncated_normal(
        [vocab_size, embedding_size], stddev=1.0/math.sqrt(embedding_size)))
    b_out = tf.Variable(tf.zeros([vocab_size]))

    nce_loss = tf.nn.nce_loss(weights=w_out, biases=b_out, labels=correct,
                              inputs=embed, num_sampled=FLAGS.num_sumpled,
                              num_classes=vocab_size)
    loss = tf.reduce_mean(nce_loss)

    global_step = tf.Variable(0, name='global_step', trainable=False)
    train_op = tf.train.GradientDescentOptimizer(FLAGS.learning_rate)\
        .minimize(loss, global_step=global_step)

    init = tf.global_variables_initializer()
    saver = tf.train.Saver(max_to_keep=3)

    with tf.Session() as sess:

        ckpt_state = tf.train.get_checkpoint_state(FLAGS.log_dir)
        if ckpt_state:
            last_model = ckpt_state.model_checkpoint_path
            saver.restore(sess,last_model)
            print("model was loaded:", last_model)
        else:
            sess.run(init)
            print("initialized.")

        last_step = sess.run(global_step)
        average_loss = 0
        for i in range(FLAGS.num_step):

            step = last_step + i + 1
            batch_inputs, batch_labels = data.create_next_batch(
                batch_size, FLAGS.num_skips, FLAGS.skip_window)
            feed_dict = {inputs: batch_inputs, correct: batch_labels}

            _, loss_val = sess.run([train_op, loss], feed_dict=feed_dict)
            average_loss += loss_val

            if step % 100 == 0:
                average_loss /= 100
                print('Average loss at step ', step, ': ', average_loss)
                average_loss = 0
                saver.save(sess, FLAGS.log_dir+'my_model.ckpt', step)




if __name__ == '__main__':
    tf.app.run()

ImportError: No module named 'data_set'